In [ ]:
import torch
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("../data/processed/dataset.csv")

# Division of the dataset into training and testing
train_data = df.sample(frac=0.8, random_state=0)
test_data = df.drop(train_data.index)

